Some thoughts

0) By scrolling down it's easy to see that at this stage I didn't get good results. [Disclaimer: Also, I know that I haven't performed sufficient tests of data before going for linear regression... And I know, I got a bit lost.]

1) Plenty of data is missing, but let's see whether for certain ids we can get full(er?... at least) dataset. In other words: maybe for all available set data is missing, but if we take each second/third/fourth id + data we'll get something valuable.

2) Trying predictions with all these features is pointless (correlations), it's reasonable to choose some more valuable features

Ad.1)
Let's start and see how many entries do we have.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

with pd.HDFStore('../input/train.h5') as train:
    df = train.get('train')
print(df.shape)

Quite a many. Now, let's see how many different ID's and timestamps do we have, e.g. let's check whether ids and timestamps are unique and are they missing anytime in data set at all.

In [ ]:
print(df.count(axis=0))
plt.plot(df.y)

My general findings after going through dataset are:
 - I think these are returns on a portfolio built out of instruments with given id (in time, -> timestamps)
 - Range is circa between -5% to 10% which seems feasible.
 - For given id of instruments we have a given set of features and timestamps. We can use these features over time to predict y_id (y for a given id) --> I should try dealing with time series!

I make a hypothesis that's it's possible to predict y by constructing y_general depending on y_ids. Y_ids, returns for a given instruments, are dependent on certain features for them.  I focus below on trying linear regression for a one given Y_id (I ignore timestamps, basically; I think now it might be better to try using time series)

In [ ]:
'''
Check how many unique id's do we have.
'''
len(df.id.unique())
'''
Soo we have 1424 different IDs. Legit. Now we'll make an array of them for future use.
'''
id_list = df.id.unique()

In [ ]:
id_df = df.loc[df['id'] == id_list[70]]
'''
prind(id_df)
From printing frame we see that for a given id we have a complete list of timestamps
'''
print(id_df.count(axis=0))

We can sort our dataframe by an ID. Then we will see complete list of consecutive timestamps and also we can notice that for a given ID, certain number of features have full data set, while some are completely absent (equal 0). We can say then that for this given instrument (with a given id) we can skip certain feature since it apparently had no value for our predictions.

For the purpose of experiment :) let's use only these features whose total nonNaN number equals number of ids/timestamps.

In [ ]:
new_id_df = id_df.dropna(axis=1, how='any')
# print(new2_id_df.count(axis=0))

Y_train = new_id_df.y
Y_train = Y_train.reset_index()
Y_train = Y_train.iloc[:,1]
print(Y_train)

In [ ]:
print(new_id_df)

In [ ]:
print(new_id_df.shape)

At this stage I think it makes sense to check correlations between the features.

In [ ]:
'''
Correlations: 
'''

c = new_id_df.corr().abs()

s = c.unstack()
so = s.order(ascending=False)
ff = pd.DataFrame(so, columns=['coeff'])
ff = pd.DataFrame(so, columns=['coeff'])
ff = ff.dropna()
ff = ff.tail(20)
ff = ff.iloc[::2]
print(ff)

In [ ]:
raw_features_list = []
for i in range (0, len(ff)):
    if any('y' in code for code in ff.iloc[[i]].index.tolist()):
        pass
    else:
        for j in range (0,1):
            raw_features_list.extend(ff.iloc[[i]].index.tolist()[j])
features_list = list(set(raw_features_list))
print(features_list)

In [ ]:
# So now we have much smaller list of features. Let's X set [we created Y beforehand]
X_train = new_id_df[features_list]
X_train = X_train.reset_index()
X_train = X_train.iloc[:,1:]
print(X_train)



In [ ]:
from sklearn import linear_model

# Split the data into training/testing sets
X_train = X_train[:-(int(len(X_train)/2))]
X_train_test = X_train[-(int(len(X_train)/2)):]

# Split the targets into training/testing sets
Y_train = Y_train[:-(int(len(Y_train)/2))]
Y_train_test = Y_train[-(int(len(Y_train)/2)):]


In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, Y_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(X_train_test) - Y_train_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_train_test, Y_train_test))

# Plot outputs
plt.plot(X_train_test, regr.predict(X_train_test), color='blue',
         linewidth=3)

plt.show()

Variance score 0.03. OK it's not the best way. Attempts to be continued.

In [ ]:
plt.plot(Y_train_test)